In [39]:
#libraries needed
import cv2
import pytesseract
import re
import os
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment

In [40]:
#extract the image to text
def extract_image_to_text(image_path):    
    global extracted_text

    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    extracted_text = pytesseract.image_to_string(gray_image)

In [41]:
#regEx patterns
def details_regEx_patterns():
    global bank_found, dates_found, amounts_found, payer_name_found, cuit_found, proof_number_found

    bank_found = re.findall(bank_pattern, extracted_text, re.IGNORECASE)
    
    dates_found = re.findall(date_pattern, extracted_text)
    amounts_found = re.findall(amount_pattern, extracted_text)
    payer_name_found = re.findall(payer_name_pattern , extracted_text)
    cuit_found = re.findall(cuit_pattern, extracted_text)
    proof_number_found = re.findall(proof_number_pattern, extracted_text)

    return bank_found, dates_found, amounts_found, payer_name_found, cuit_found, proof_number_found

In [42]:
# Extract the first date, amount, and CUIT number if any are found
def extract_details():
    global bank, date, amount, payer, cuit, proof_number
    
    bank = bank_found[0] if bank_found else None
    date = dates_found[0] if dates_found else None
    amount = amounts_found[0] if amounts_found else None
    proof_number = proof_number_found[0] if proof_number_found else None
    
    payer = payer_name_found[0] if payer_name_found else None
    cuit = cuit_found[0] if cuit_found else None

In [43]:
#return the extracted details
def get_extracted_details(bank, date, amount, payer, cuit, proof_number):
    return {
        'BANCO': bank,
        'FECHA': date,
        'IMPORTE': amount,
        'TITULAR': payer,
        'CUIT': cuit,
        'NRO COMPROBANTE': proof_number
    }

In [44]:
#new Excel workbook
wb = Workbook()
ws = wb.active
ws.title = "Extracted Data"

# Add headers to the Excel sheet
headers = [ 'FECHA', 'IMPORTE', 'NRO COMPROBANTE', 'TITULAR', 'CUIT', 'BANCO']

ws.append(headers)

# Apply bold style to the header cells
bold_font = Font(bold=True)
for cell in ws[1]:
    cell.font = bold_font

# Apply padding to all cells
for row in ws.iter_rows():
    for cell in row:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

#check if image exists and adjust the padding
def check_image_and_padding(folder_path):
# Process each image in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
            # Extract data from the image
            image_path = os.path.join(folder_path, filename)
            extracted_data = extract_data_from_image(image_path)
    
            # Add the extracted data to the Excel sheet
            # row_data = [extracted_data[header] for header in headers]
            row_data = [extracted_data.get(header) for header in headers]
            ws.append(row_data)
    
    # Automatically adjust column widths based on content
    for column_cells in ws.columns:
        max_length = 0
        for cell in column_cells:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2) * 1.2  # Adjust the multiplier as needed
        ws.column_dimensions[column_cells[0].column_letter].width = adjusted_width

# till now banks

In [45]:
#bancopatagonia

def extract_data_from_image(image_path):

    global bank_pattern, date_pattern, amount_pattern, proof_number_pattern, payer_name_pattern, cuit_pattern

    extract_image_to_text(image_path)
         
    lines = extracted_text.split('\n')
    lines_12 = lines[11]

    # # regEx patterns for different types of data to extract
    bank_pattern = 'bancopatagonia'

    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
    amount_pattern = r'\$\s*\d[\d,\.]*'
    proof_number_pattern = r'\b\d{10}\b'
    
    payer_name_pattern = lines_12

    cuit_pattern =  'None' #MARTIN said Cuit IS FILLED MANUALLY 


    # Extract information using the regEx patterns
    details_regEx_patterns()

    # Extract the date, amount, and CUIT number if any are found
    extract_details()

    #this is important to extract the [1] amount coz the fun has [0] default
    amount = amounts_found[1] if amounts_found else None

    # Return the extracted details
    return get_extracted_details(bank, date, amount, payer, cuit, proof_number)

# Define the folder containing the images
folder_path = 'bancopatagonia/bancopatagonia_assets'

#function 
check_image_and_padding(folder_path)

# Save the Excel file
# wb.save('cccc.xlsx')

# print('Everything is fine!')


In [46]:
#galicia
def extract_data_from_image(image_path):

    global bank_pattern, date_pattern, amount_pattern, proof_number_pattern, payer_name_pattern, cuit_pattern

    extract_image_to_text(image_path)

    lines = extracted_text.strip().split('\n')
    line_9 = lines[8]

    # Define regular expression patterns for different types of data to extract

    bank_pattern = 'galicia'
    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
    amount_pattern = r'\$\s*\d[\d,\.]*'
    proof_number_pattern = r'\b\d{11}\b'
    
    payer_name_pattern = line_9 
    
    cuit_pattern =  r'\b\d{2}-\d{8}-\d{1}\b'


    # Extract information using the regEx patterns
    details_regEx_patterns()

    # Extract the date, amount, and CUIT number if any are found
    extract_details()

    # return extracted_details
    return get_extracted_details(bank, date, amount, payer, cuit, proof_number)

# Define the folder containing the images
folder_path = 'galicia/galicia_assets'

#function 
check_image_and_padding(folder_path)

# Save the Excel file
# wb.save('galicia_extracted_info.xlsx')


In [47]:
#mercado pago
def extract_data_from_image(image_path):

    global bank_pattern, date_pattern, amount_pattern, proof_number_pattern, payer_name_pattern, cuit_pattern

    extract_image_to_text(image_path)

    # Define regular expression patterns for different types of data to extract
    bank_pattern = 'mercado pago'
    
    date_pattern = r'\b\d{1,2} de [a-z]+ \d{4}\b'
    amount_pattern = r'\$\s*\d+(?:[.,]\d+)?'
    payer_name_pattern = r'(?:de )?([A-Z][a-z]+(?: [A-Z][a-z]+)*)'
    cuit_pattern = r'\b\d{2}-\d{8}-\d{1}\b'
    proof_number_pattern = r'\b\d{11}\b'

    # Find all occurrences of dates, amounts, and CUIT numbers in the extracted text
    details_regEx_patterns()

    # Extract the date, amount, and CUIT number if any are found
    extract_details()

    #this is important to extract the [1] amount coz the fun has [0] default
    payer = payer_name_found[1] if payer_name_found else None

    # Return the extracted information as a dictionary
    return get_extracted_details(bank, date, amount, payer, cuit, proof_number)

# Define the folder containing the images
folder_path = 'mercado_pago/mercado_pago_assets'

#function 
check_image_and_padding(folder_path)

# Save the Excel file
# wb.save('mercado_pago_extracted_info.xlsx')

In [48]:
#santander
def extract_data_from_image(image_path):

    global bank_pattern, date_pattern, amount_pattern, proof_number_pattern, payer_name_pattern, cuit_pattern
    extract_image_to_text(image_path)

    # Define regular expression patterns for different types of data to extract
    bank_pattern = 'santander'
    
    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
    amount_pattern = r'\$\s*\d[\d,\.]*'
    proof_number_pattern = r'\b\d{8}\b'
    
    payer_name_pattern = 'None' #MARTIN SAID NAME IS FILLED MANUALLY 
    cuit_pattern =  'None' #MARTIN SAID Cuit IS FILLED MANUALLY 

        # Extract information using the regEx patterns
    details_regEx_patterns()

    # Extract the date, amount, and CUIT number if any are found
    extract_details()

    # Return the extracted details
    return get_extracted_details(bank, date, amount, payer, cuit, proof_number)

# Define the folder containing the images
folder_path = 'santander/santander_assets'

#function 
check_image_and_padding(folder_path)

# Save the Excel file
# wb.save('santander_extracted_info.xlsx')

In [49]:
#supervielle
def extract_data_from_image(image_path):

    global bank_pattern, date_pattern, amount_pattern, proof_number_pattern, payer_name_pattern, cuit_pattern
    extract_image_to_text(image_path)

    # Define regular expression patterns for different types of data to extract
    bank_pattern = 'supervielle'
    
    date_pattern = r'\b\d{1,2}\s+[A-Za-z]+\s+\d{4}\b'
    amount_pattern = r'\$\s*\d[\d,\.]*'
    proof_number_pattern = r'\b\d{4}\b' #have to ask martin about this coz there's none
    
    payer_name_pattern = 'None' #MARTIN said NAME IS FILLED MANUALLY 
    cuit_pattern =  'None' #MARTIN said Cuit IS FILLED MANUALLY 

    # Extract information using the regEx patterns
    details_regEx_patterns()

    # Extract the date, amount, and CUIT number if any are found
    extract_details()

    # Return the extracted details
    return get_extracted_details(bank, date, amount, payer, cuit, proof_number)

# Define the folder containing the images
folder_path = 'supervielle/supervielle_assets'

#function 
check_image_and_padding(folder_path)

# Save the Excel file
# wb.save('supervielle_extracted_info.xlsx')

# new banks

In [50]:
#bna  -- full details are yet to be known
def extract_data_from_image(image_path):

    global bank_pattern, date_pattern, amount_pattern, proof_number_pattern, payer_name_pattern, cuit_pattern
    extract_image_to_text(image_path)

    # Define regular expression patterns for different types of data to extract
    bank_pattern = 'bna'
    
    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
    amount_pattern = r'\$\s*\d[\d,\.]*'
    proof_number_pattern = r'\b\d{8}\b' #have to ask martin about this coz there's none
    
    payer_name_pattern = 'None' #MARTIN said NAME IS FILLED MANUALLY 
    cuit_pattern =  r'\b\d{11}\b'  #MARTIN said Cuit IS FILLED MANUALLY 

    # Extract information using the regEx patterns
    details_regEx_patterns()

    # Extract the date, amount, and CUIT number if any are found
    extract_details()

    # Return the extracted details
    return get_extracted_details(bank, date, amount, payer, cuit, proof_number)

# Define the folder containing the images
folder_path = 'bna/bna_assets'

#function 
check_image_and_padding(folder_path)

# Save the Excel file
# wb.save('supervielle_extracted_info.xlsx')

In [51]:
#cuenta_dni  -- full details are yet to be known
def extract_data_from_image(image_path):

    global bank_pattern, date_pattern, amount_pattern, proof_number_pattern, payer_name_pattern, cuit_pattern
    extract_image_to_text(image_path)

    lines = extracted_text.split('\n')
    lines_8 = lines[7]

    # Define regular expression patterns for different types of data to extract
    bank_pattern = 'Cuenta DNI'
    
    date_pattern = r'\b\d{1,2}/\d{1,2}/\d{4}\b'
    amount_pattern = r'\$\s*\d[\d,\.]*'
    proof_number_pattern = r'\b\d{6}\b' #have to ask martin about this coz there's none
    
    payer_name_pattern = lines_8
    
    cuit_pattern =  r'\b\d{11}\b' 

    # Extract information using the regEx patterns
    details_regEx_patterns()

    # Extract the date, amount, and CUIT number if any are found
    extract_details()
    bank = bank_pattern

    # Return the extracted details
    return get_extracted_details(bank, date, amount, payer, cuit, proof_number)

# Define the folder containing the images
folder_path = 'cuenta_dni/cuenta_dni_assets'

#function 
check_image_and_padding(folder_path)

# Save the Excel file
# wb.save('supervielle_extracted_info.xlsx')

# saving and opening

In [52]:

# Specify the path to the extracted file
extracted_file_path = 'extracted_info.xlsx'

# Save the Excel file
wb.save(extracted_file_path)

In [ ]:
import subprocess
import os

#if you are on linux 
# Open the file with LibreOffice in linux
subprocess.run(['libreoffice', extracted_file_path])

# Open the file with Microsoft Excel in windows
#os.system("start EXCEL.EXE extracted_info.xlsx")
